In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from rts.db_settings import DATABASE_URL
from rts.db.dao import DataAccessObject
from rts.pipeline import PipelineIOC

DataAccessObject().connect(DATABASE_URL)


In [3]:
metadata = '../data/metadata.hdf5'
with pd.HDFStore(metadata) as store:
    keys = store.keys()
    df = store.get(keys[0])  

In [17]:
# This preprocessing step should be outside of the pipeline code and come with the dataframe
def find_mp4_files(root_folder):
    import os
    mp4_files = []

    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.mp4'):
                mp4_files.append(os.path.join(foldername, filename))

    return mp4_files

mp4_files = find_mp4_files("/media/data/ioc/videos/")
file_map = {}
for mp4 in mp4_files:
    file_map[mp4.split('/')[-1].split('.')[0]] = mp4

df['path'] = df['guid'].apply(lambda x: file_map[x] if x in file_map else None)

In [6]:
# we need a path for each video, otherwise we can't process
df = df[df['path'].notna()]

In [ ]:
PipelineIOC().ingest(df, notebook=True)

In [ ]:
# Process a single video and upload clips. This funcction would also take a dataframe that has more than one video
PipelineIOC().ingest(df[df.guid == 'MDA42810489'], notebook=True)

In [16]:
def find_mp4_files(root_folder):
    import os
    mp4_files = []

    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            # if filename.endswith('.mp4'):
            mp4_files.append(os.path.join(foldername, filename))

    return mp4_files

metadata_files = find_mp4_files("/media/data/ioc/metadata/")
file_map = {}
for mp4 in mp4_files:
    file_map[mp4.split('/')[-1].split('.')[0]] = mp4

In [55]:
for mp4 in mp4_files:
    if 'MDA42710309' in mp4:
        print(mp4)  

/media/data/ioc/metadata/9/0/20230512115012_OCS317087_MDA42710309.xml


In [12]:
for mp4 in mp4_files:
    if 'MDA42710309' in mp4:
        print(mp4)  

/media/data/ioc/videos/MP4_PROXY_HR/MDA42710309.LR.mp4


In [18]:
len(metadata_files), len(mp4_files)

(22348, 31304)

In [20]:
# clean up metadata and mp4 file IDs
metadata_files = [x.split('/')[-1] for x in metadata_files]
metadata_files.sort()
mp4_files = [x.split('/')[-1] for x in mp4_files]
mp4_files.sort()
mp4_files = [x.split('.')[0] for x in mp4_files]
metadata_files = [x.split('.')[0] for x in metadata_files]
metadata_files = [x.split('_')[-1] for x in metadata_files]
# count overlap between metadata and mp4 files
len(set(metadata_files).intersection(set(mp4_files)))